<a href="https://colab.research.google.com/github/hood-boi/world-news-chatbot/blob/master/Data_Collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langdetect

     |████████████████████████████████| 1.0MB 2.8MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.7-cp36-none-any.whl size=993460 sha256=cd38295653301479d01597d07e0aabf421ed635533e982ee0621bf6645940bb7
  Stored in directory: /root/.cache/pip/wheels/ec/0c/a9/1647275e7ef5014e7b83ff30105180e332867d65e7617ddafe
Successfully built langdetect


In [2]:
import requests
import json
import time
import string
import nltk
from langdetect import detect
nltk.download('words')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [0]:
printable = set(string.printable)

def cleanString(text):
    printable = set(string.printable)
    text_i = ''.join(filter(lambda x: x in printable, text))
    return text_i;

In [4]:
! wc '/content/gdrive/My Drive/Chatbot/rworldnews.txt'

wc: '/content/gdrive/My Drive/Chatbot/rworldnews.txt': No such file or directory


In [5]:
subreddit = "todayilearned"

def updateListingDict(postsJSON, postsDICT):
    chunk_size = len(postsJSON["data"]["children"])
    posts = postsJSON["data"]["children"]
    
    for i in range(chunk_size): #Loop through each reddit post
        r_post_title = posts[i]["data"]["title"];
        r_post_title = " ".join(r_post_title.split());
        r_post_title = r_post_title.replace('\n', ' ');
        r_post_title = r_post_title.replace('\t', ' ');
        r_post_title = r_post_title.replace('\'', '');
        r_post_title = r_post_title.replace('\"', '');

        postsDICT[posts[i]["data"]["id"]] = cleanString(r_post_title);

after = '';
listing_dict = {};

for i in range(0, 10):
    if(after):
        print("[AFTER] = ", after)
        r_worldnews = requests.get('https://www.reddit.com/r/'+ subreddit +'/top.json?t=all&limit=100&after='+after, 
                                headers = { 'User-agent' : 'ChatBot', 
                                            'Accept' : '*/*',
                                            'Cache-Control' : 'no-cache'
                                          });
        if(r_worldnews.status_code == 200):
            postsJSON = r_worldnews.json()
            updateListingDict(postsJSON, listing_dict);
            after = postsJSON["data"]["after"];
    else:
        print("[AFTER] = NONE")
        r_worldnews = requests.get('https://www.reddit.com/r/'+ subreddit +'/top.json?t=all&limit=100', 
                                headers = { 'User-agent' : 'ChatBot', 
                                            'Accept' : '*/*',
                                            'Cache-Control' : 'no-cache' 
                                          });
        if(r_worldnews.status_code == 200):
            postsJSON = r_worldnews.json()
            updateListingDict(postsJSON, listing_dict);
            #print("[GET] request successful")
            after = postsJSON["data"]["after"];
            
            

[AFTER] = NONE
[AFTER] =  t3_an3th5
[AFTER] =  t3_96h90n
[AFTER] =  t3_7qt84c
[AFTER] =  t3_ajga93
[AFTER] =  t3_86cmhd
[AFTER] =  t3_81awfz
[AFTER] =  t3_60n7wp
[AFTER] =  t3_5lauqm
[AFTER] =  t3_5u9fji


In [6]:
print(len(listing_dict))

for key, value in listing_dict.items():
    print(key, value)
    break;

989
7pbzcb TIL After Col. Shaw died in battle, Confederates buried him in a mass grave as an insult for leading black soldiers. Union troops tried to recover his body, but his father sent a letter saying We would not have his body removed from where it lies surrounded by his brave and devoted soldiers.


In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
file_handle = open('/content/gdrive/My Drive/Chatbot/r'+subreddit+'.txt', 'a+')

#Expecting array of JSON children
def recursePostComments(context, commentsJSON):
   
    size = len(commentsJSON)
    unique_context = 1;
    for i in range(0, size):
        #print(commentsJSON[i])
        if('data' in commentsJSON[i].keys()):
            if('body' in commentsJSON[i]['data'].keys()):
                reply = commentsJSON[i]['data']['body'];
                if("[deleted]" in reply or "[removed]" in reply):
                    continue;
                else:
                    reply = reply.replace('\n', ' ');
                    reply = reply.replace('\t', ' ');
                    reply = reply.replace('\'', '');
                    reply = reply.replace('\"', '');
                    reply = reply.replace('&gt', '');
                    reply = reply.replace('&lt', '');
                    #print("Context : ", context);
                    
                    if (len(reply) > 500):
                        continue;
                        #print("\tLen :", len(reply));
                    #    t_reply = summarizeText(reply);
                    #    if (len(t_reply) > 0):
                    #        reply = t_reply;
                    #print("\tReply :", reply);
                    
                    reply = cleanString(reply);
                    reply = " ".join(reply.split());
                                        
                    
                    if(reply.isspace() or context.isspace()):
                        print("Error : ", reply, ", ", context)
                        continue;                    
                    else:
                        if(unique_context == 1):
                            line_to_write = "{}\t{}\n".format(reply.strip(), context.strip());
                            file_handle.write(line_to_write)
                            unique_context = 0;
                    
                    if('replies' in commentsJSON[i]['data'].keys()):
                        if(commentsJSON[i]['data']['replies']):
                            if('data' in commentsJSON[i]['data']['replies'].keys()):
                                if(commentsJSON[i]['data']['replies']['data']):
                                    if('children' in commentsJSON[i]['data']['replies']['data'].keys()):
                                        recursePostComments(reply, commentsJSON[i]['data']['replies']['data']['children']);
            else:
                continue;
        else:
            continue;
    return;


In [9]:
! wc '/content/gdrive/My Drive/Chatbot/rworldnews.txt'

  248952 11261157 65389928 /content/gdrive/My Drive/Chatbot/rworldnews.txt


In [0]:
for key, value in listing_dict.items():
    print(key, value)
    time.sleep(2)
    r_comments = requests.get('https://www.reddit.com/r/'+ subreddit +'/comments/' + key +'.json?limit=1000&sort=confidence&depth=3', 
                                headers = { 'User-agent' : 'ChatBot', 
                                            'Accept' : '*/*',
                                            'Cache-Control' : 'no-cache' 
                                          });
    r_comments_json = r_comments.json();
    context = value;
    commentsJSON = r_comments_json[1]["data"]["children"]
    recursePostComments(context, commentsJSON)
    
file_handle.close()

7pbzcb TIL After Col. Shaw died in battle, Confederates buried him in a mass grave as an insult for leading black soldiers. Union troops tried to recover his body, but his father sent a letter saying We would not have his body removed from where it lies surrounded by his brave and devoted soldiers.
8ih4tq TIL that in 1916 there was a proposed Amendment to the US Constitution that would put all acts of war to a national vote, and anyone voting yes would have to register as a volunteer for service in the United States Army.
b0fz4k TIL of Dr. Donald Hopkins. He helped eradicate Smallpox, and is on the verge of killing another disease. Hes taken Guinea Worm Disease down from 3.5 million cases a year to just 28 cases last year.
93dfyx TIL A Japanese company has awarded its non-smoking employees 6 extra vacation days to compensate for the smokers smoke breaks
7yqmzl TIL when the Nazis burned Sigmund Freuds books he said, What progress we are making. In the Middle Ages they would have burned 

In [0]:
#https://stackoverflow.com/questions/8689795/how-can-i-remove-non-ascii-characters-but-leave-periods-and-spaces-using-python
    

In [0]:
import string
print(string.printable)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	



In [0]:
! cd '/content/gdrive/My Drive/Chatbot/' ; ls r*.txt | xargs -L 1 wc

  325392 16109903 87161691 rAskReddit.txt
  337107 15033769 85689554 rnews.txt
  339541 12756240 72687966 rnottheonion.txt
   68529  3350392 19319193 routoftheloop.txt
  248952 11261157 65389928 rworldnews.txt
